In [11]:
import numpy as np
from astropy import units as u
from astropy import constants as con
%config Completer.use_jedi = False

In [41]:
l_spec = 50 * u.imperial.ft
l_meas = 15.24 * u.m
z0_spec = 50 * u.ohm
z0_meas = 58 * u.ohm
vf = .8035
freq = 14 * u.MHz

In [42]:
(l_spec - l_meas).to(u.imperial.inch)

<Quantity 8.52651283e-14 inch>

In [43]:
def get_input_z(zL, z0, l, vf, freq):
    gamma = 1j* 2 * np.pi * freq / (vf * con.c)

    B = np.tanh((gamma * l).decompose().value)
    z_in = z0 * (zL + B * z0) / (z0 + B * zL)
    return z_in

zL = 75 * u.ohm
z0 = z0_meas
l = l_meas
get_input_z(zL, z0, l, vf, freq)#.decompose()

<Quantity 58.10174235+14.96245398j Ohm>

In [44]:
def get_input_z_from_fundamental(l, zL, Lp, Cp, Rp, Gp, freq):
    w = 2 * np.pi * freq
    
    term1 = Rp + 1j * w * Lp
    term2 = Gp + 1j * w * Cp
    
    gamma = np.sqrt(term1 * term2)
    zI = np.sqrt(term1 / term2)
    
    trig = np.tanh(gamma * l)
    
    z_in = zI * (zL + zI * trig) / (zI + zL * trig)
    return z_in

In [ ]:
def penalty(p, zL):
    z_in = get_input_z_from_fundamental(p.l, zL, p.Lp, p.Cp, p.Rp, p.Gp, freq)
    z_meas = p.x
    delta = z_meas - z_in
    return np.sum(np.conjugate(delta) * delta)
